## Test GAIA DR2

Test to search for specific data in DR2. <br>
Note that gaia-on-tap is by default for DR1 <br>

In [1]:
import sys, os
sys.path.append('../../src')

import astropy.coordinates as coord
import matplotlib.pyplot as plt
from pylab import rcParams
import pandas as pd
from gaia.tap import cone_search


import gaia_utils as gu

%matplotlib inline

## directory
rootdir = "/home/stephane/Science/GAIA"
wdir    = "%s/products"%(rootdir)
datadir = "%s/master/notebooks/data"%(rootdir)

os.chdir(wdir)

### OC selection 

In [2]:
def plotCluster(cluster_candidates, clustername, display = True):

    figname = "%s-gaia.png"%(clustername)
    
    rcParams['figure.figsize'] = 14, 21
    f, axarr = plt.subplots(3, 2)

# ax.scatter(cluster_candidates["ra"], cluster_candidates["dec"], s=1, c="#000000")
    axarr[0,0].scatter(cluster_candidates["ra"], cluster_candidates["dec"], s=1, c="#000000")
    axarr[0,0].set_xlabel(r"$\alpha$")
    axarr[0,0].set_ylabel(r"$\delta$")

    axarr[0,1].scatter(cluster_candidates["l"], cluster_candidates["b"], s=1, c="#000000")
    axarr[0,1].set_xlabel(r"l")
    axarr[0,1].set_ylabel(r"b")

    axarr[1,0].scatter(cluster_candidates["l"], cluster_candidates["parallax"], s=1, c="#000000")
    axarr[1,0].set_xlabel(r"l")
    axarr[1,0].set_ylabel(r"p (mas)")
    axarr[1,0].set_ylim([-2,4])

    axarr[1,1].scatter(cluster_candidates["ra"], cluster_candidates["pmra"], s=1, c="#000000")
    axarr[1,1].set_xlabel(r"$\alpha$")
    axarr[1,1].set_ylabel(r"PM RA (mas/yr)")
    axarr[1,1].set_ylim([-30,30])

    axarr[2,0].scatter(cluster_candidates["pmdec"], cluster_candidates["pmra"], s=1, c="#000000")
    axarr[2,0].set_xlabel(r"PM DEC (mas/yr)")
    axarr[2,0].set_ylabel(r"PM RA (mas/yr)")
    axarr[2,0].set_xlim([-40,10])
    axarr[2,0].set_ylim([-30,30])

    axarr[2,1].scatter(cluster_candidates["parallax"], cluster_candidates["pmra"], s=1, c="#000000")
    axarr[2,1].set_xlabel(r"p (mas)")
    axarr[2,1].set_ylabel(r"PM RA (mas/yr)")
    axarr[2,1].set_xlim([-2,4])
    axarr[2,1].set_ylim([-30,30])

    f.subplots_adjust(hspace=0.5)
    
    plt.savefig(wdir+"/"+figname)
    if display:
        plt.show()

In [ ]:
## read the cluster list from HEASARC
def read_cluster_list(filelist):
    
    df = pd.read_csv(filelist, sep='|')
    
    return(df)

## write in fileoutGaia the selected cluster for GAIA
def write_SCgaia(fileout, sc):
    
    with open(fileout,"a") as f:
        f.write("%s \n"%(sc))
        
## find the last SC found.
def find_lastSC(filegaia):
    if os.path.exists(filegaia):
        with open(filegaia) as f:
            lines = f.read().splitlines()
        last_line = lines[-1]
        
    return(last_line)

### Main ..

Making the plots and analysis of the targets.

In [ ]:
# filelist = datadir+"/"+"BrowseTargets.19060.1528915517"
filelist = datadir+"/"+"BrowseTargets.13175.1528937650"
filelist = datadir+"/"+"BrowseTargets.31265.1529958911"

fileoutGaia = wdir + "/"+"BrowseTargets.31265.1529958911.gaia.selected.txt"

# cluster = coord.SkyCoord.from_name("M67")
# cluster = coord.SkyCoord.from_name("NGC2287")
# cluster = coord.SkyCoord.from_name("Collinder 65")

df_cluster = read_cluster_list(filelist)
print(df_cluster.index)
print(df_cluster.columns)
    
lastSC = find_lastSC(fileoutGaia)

for index, row in df_cluster.iterrows():
    clustername = row['name'].strip()
    print("Cluster: %s"%(clustername))
    
    # c = coord.SkyCoord.from_name(clustername)
    rasplit = row['ra'].split(' ')
    decsplit = row['dec'].split(' ')
    racluster = "%sh%sm%ss"%(rasplit[0],rasplit[1],rasplit[2])
    deccluster = "%sd%sm"%(decsplit[0],decsplit[1])
    c = coord.SkyCoord(racluster, deccluster, frame='icrs')
    
    # Get everything within 1 degree radius of the cluster.
    cluster_candidates = cone_search(c.ra.deg, c.dec.deg, 0.5, table="gaiadr2.gaia_source")
    gaia = gu.gaiaSet(cluster_candidates)
    selected = gaia.isHomogeneous(tol = 0.02)
    if selected:
        write_SCgaia(fileoutGaia,clustername)
        print("## Selected")
    else:
        print("## Not selected")
        
    plotCluster(cluster_candidates, clustername, display = True)


    if len(cluster_candidates) == 0:
        print("## No data...")


RangeIndex(start=0, stop=1296, step=1)
Index(['Unnamed: 0', 'name', 'ra', 'dec', 'cluster_radius', 'rad_vel ',
       'rad_vel_error', 'num_cluster_stars', 'distance', 'log_age',
       'log_age_error', 'class            ', 'Unnamed: 12'],
      dtype='object')
FSR 1577 
Cluster: MWSC 4688


### Plots ...